In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories

In [8]:
config = CONFIG_FILE_PATH
print(config)
yaml = read_yaml_file(config)
print(yaml)

config\config.yaml
[2024-01-03 16:20:33,634 - INFO - common - YAML file loaded successfully: config\config.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://storage.googleapis.com/heispv/pizza-steak-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'base_model_preparation': {'root_dir': 'artifacts/base_model_preparation', 'base_model_path': 'artifacts/base_model_preparation/base_model.h5', 'updated_base_model_path': 'artifacts/base_model_preparation/updated_base_model.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_log_dir': 'artifacts/prepare_callbacks/tensorboard_logs', 'checkpoint_path': 'artifacts/prepare_callbacks/checkpoint_dir/checkpoint.h5'}}


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = str(config.source_url),
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

        return data_ingestion_config

In [10]:
config = ConfigurationManager()

[2024-01-03 16:21:02,951 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-03 16:21:02,955 - INFO - common - YAML file loaded successfully: params.yaml]
[2024-01-03 16:21:02,957 - INFO - common - Directory created: artifacts]


In [12]:
data_config = config.get_data_ingestion_config()

[2024-01-03 16:21:48,113 - INFO - common - Directory created: artifacts/data_ingestion]


In [14]:
data_config.root_dir

WindowsPath('artifacts/data_ingestion')

In [10]:
import os
import urllib.request as request
import zipfile
from PizzaSteakClassifier import logger
from PizzaSteakClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        """
        Downloads the data from the specified source URL and saves it locally.

        If the local data file already exists, it logs a message indicating its size.
        If the local data file does not exist, it downloads the file and logs the download details.

        Returns:
            None
        """
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {file_name} of size {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"Data file {self.config.local_data_file} already exists with the size {get_size(Path(self.config.local_data_file))}")

    def unzip_data(self):
            """
            Unzips the data file to the specified directory.

            Args:
                self.config.local_data_file (str): The path to the local data file.
                self.config.unzip_dir (str): The directory where the data file will be extracted.

            Returns:
                None
            """
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise(e)

[2023-12-30 16:36:38,076 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2023-12-30 16:36:38,079 - INFO - common - YAML file loaded successfully: params.yaml]
[2023-12-30 16:36:38,081 - INFO - common - Directory created: artifacts]
[2023-12-30 16:36:38,082 - INFO - common - Directory created: artifacts/data_ingestion]
[2023-12-30 16:36:38,084 - INFO - 3833457559 - Data file artifacts/data_ingestion/data.zip already exists with the size 106860.9385 KB]
